In [16]:
from selenium import webdriver
import json
import pandas as pd
import datetime
import time
from requests_oauthlib import OAuth1Session
import sys

In [17]:
def get_table():
    #アカウント情報を取得
    f = open("account.json","r")
    account = json.load(f)
    f.close()

    #ログイン処理
    driver = webdriver.Chrome()
    driver.get("https://www.k.kyoto-u.ac.jp/student/la/notice/cancel")
    driver.find_element_by_id("username").send_keys(account["ecs-id"])
    driver.find_element_by_id("password").send_keys(account["password"])
    driver.find_element_by_name("_eventId_proceed").click()


    #DataFrameを作成
    df = pd.DataFrame()

    #テーブルデータを取得
    elements = driver.find_elements_by_class_name("odd_normal")
    for e in elements:
        tds = e.find_elements_by_tag_name("td")
        data = []
        for td in tds:
            data.append(str(td.text))
        tmp_df = pd.DataFrame(data)
        df = df.append(tmp_df.T)

    elements = driver.find_elements_by_class_name("even_normal")
    for e in elements:
        tds = e.find_elements_by_tag_name("td")
        data = []
        for td in tds:
            data.append(str(td.text))
        tmp_df = pd.DataFrame(data)
        df = df.append(tmp_df.T)

    driver.close()
    driver.quit()

    #データの整理
    df = df.iloc[:,0:4]
    df.columns =["time","subject","teacher","date"]
    df = df.reset_index(drop=True)
    change_time = lambda x: x[-2:-1]
    delete_time = lambda x : x[:-4]
    df["time"] = df["date"].apply(change_time)
    df["time"] = df["time"].astype(int)
    df["date"] = df["date"].apply(delete_time)
    df["date"] = pd.to_datetime(df["date"])
    df["date"] = df["date"].dt.date
    df["teacher"] = df["teacher"].str.replace("\n"," ")

    return df

In [70]:
#date日のnow限以降の休講情報のツイート文章をリストで返す
def create_messages(df,date,now):
    msgs = []
    data = []
    msg =  date.strftime("%m/%d")+"の休講情報"
    
    for i in range(now,6):   
        tmp_df = df[df["time"] == i]
        msg +="\n【{}限】\n".format(i)
        data = []
        
        for index, row in tmp_df.iterrows():
            data.append("{}({})\n".format(row["subject"],row["teacher"]))
    
        for j in range(len(data)):
            
            if len(msg) + len(data[j])  < 140:
                msg += data[j]
            else:
                msgs.append(msg)
                msg = date.strftime("%m/%d")+"の休講情報(続き)"  
                msg +="\n【{0}限】\n".format(i)
                msg += data[j]
    msgs.append(msg)
    return msgs

In [40]:
def post_to_twitter(msg):
    f = open("twitter_account.json","r")
    tw_ac = json.load(f)
    f.close()
    twitter = OAuth1Session(tw_ac['consumer_key'], tw_ac['consumer_secret'], tw_ac['access_token_key'], tw_ac['access_token_secret'])
    params = {"status": msg}

    try:
        req = twitter.post("https://api.twitter.com/1.1/statuses/update.json", params = params)
        print(req.text)
    except:
        print("can't post.")
        print(req)

In [35]:
df = get_table()

In [21]:
df.sort_values(by='date') 

,time,subject,teacher,date
0,1,図学Ａ,中嶋 節子,2018-05-11
24,1,植物自然史Ｉ,瀬戸口 浩彰,2018-05-11
27,5,ILASセミナー ：現代思想と精神分析,松本 卓也,2018-05-11
25,2,現代経済社会論Ｉ,黒宮 一太,2018-05-11
26,3,現代文明Ｉ,黒宮 一太,2018-05-11
1,1,英語リーディング,横山 仁視,2018-05-11
2,2,英語リーディング,横山 仁視,2018-05-11
3,4,現代文明Ｉ,黒宮 一太,2018-05-11
5,5,ILASセミナー ：心の発達ゼミ,船曳 康子,2018-05-14
28,5,ILASセミナー ：曲がった空間での最適化 幾何学と応用数学,佐藤 寛之,2018-05-14


In [36]:
today = datetime.date.today()
yesterday = today + datetime.timedelta(days = +4)
today_df = df[df["date"] == today ]

,time,subject,teacher,date
0,1,図学Ａ,中嶋 節子,2018-05-11
1,1,英語リーディング,横山 仁視,2018-05-11
2,2,英語リーディング,横山 仁視,2018-05-11
3,4,現代文明Ｉ,黒宮 一太,2018-05-11
24,1,植物自然史Ｉ,瀬戸口 浩彰,2018-05-11
25,2,現代経済社会論Ｉ,黒宮 一太,2018-05-11
26,3,現代文明Ｉ,黒宮 一太,2018-05-11
27,5,ILASセミナー ：現代思想と精神分析,松本 卓也,2018-05-11


In [72]:
msgs = create_messages(today_df,today,2)

for msg in msgs:
    post_to_twitter(msg)

{"created_at":"Fri May 11 14:59:20 +0000 2018","id":994955120939941888,"id_str":"994955120939941888","text":"05\/11\u306e\u4f11\u8b1b\u60c5\u5831\n\u30102\u9650\u3011\n\u82f1\u8a9e\u30ea\u30fc\u30c7\u30a3\u30f3\u30b0(\u6a2a\u5c71\u3000\u4ec1\u8996)\n\u73fe\u4ee3\u7d4c\u6e08\u793e\u4f1a\u8ad6\uff29(\u9ed2\u5bae\u3000\u4e00\u592a)\n\n\u30103\u9650\u3011\n\u73fe\u4ee3\u6587\u660e\uff29(\u9ed2\u5bae\u3000\u4e00\u592a)\n\n\u30104\u9650\u3011\n\u73fe\u4ee3\u6587\u660e\uff29(\u9ed2\u5bae\u3000\u4e00\u592a)\n\n\u30105\u9650\u3011\nILAS\u30bb\u30df\u30ca\u30fc \uff1a\u73fe\u4ee3\u601d\u60f3\u3068\u7cbe\u795e\u5206\u6790(\u677e\u672c\u3000\u5353\u4e5f)","truncated":false,"entities":{"hashtags":[],"symbols":[],"user_mentions":[],"urls":[]},"source":"\u003ca href=\"https:\/\/twitter.com\/KUBotver2\" rel=\"nofollow\"\u003eKU_Kyukou_Bot\u003c\/a\u003e","in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":nul